<h1><center>Recipe Chatbot Project</center></h1>
<h2><center>DATA-641</center></h2>
<h3><center>Sobanaa Jayakumar & Reina Villanueva-Unger</center></h3>

## 1. Setup
### A. Packages

In [38]:
import os
import pandas as pd
import numpy as np

# data processing
import nltk
from nltk.corpus import stopwords
import re

#feature engg
from sklearn.feature_extraction.text import TfidfVectorizer

## model & processing libraries
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
from sklearn import feature_selection
from sklearn.linear_model import LogisticRegressionCV
from sklearn import metrics
from sklearn import utils
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.linear_model import LogisticRegression
import pickle

## DB accesses
import sqlite3 as sq

### B. Set Paths

i. Data Paths

In [2]:
DATA_CUISINE_PATH = "data/cuisine_data/"
DATA_RECIPES_PATH = "data/recipes_data/"

ii. Model Paths

In [3]:
MODEL_PATH = "models/nlp"
MODEL_EMBEDDINGS_PATH = os.path.join(MODEL_PATH, 'similarity_embeddings')

### C. Data Import Functions

In [55]:
# Cuisine Data
def import_cuisine_data(): # import_data()
    train = pd.read_json(os.path.join(DATA_CUISINE_PATH, 'train.json'))
    return train
    #test = pd.read_json(os.path.join(DATA_CUISINE_PATH, 'test.json'))
    #return pd.concat([train,test],axis=0)

# Recipe Data 

def import_recipe_data():
    all_recipes = pd.read_json('./data/recipes_data/recipes_raw_nosource_allrecipes.json', orient='index')
    epicurious = pd.read_json('./data/recipes_data/recipes_raw_nosource_epicurious.json', orient='index')
    food_network = pd.read_json('./data/recipes_data/recipes_raw_nosource_foodnetwork.json', orient='index')
    data = pd.concat([all_recipes, epicurious, food_network], axis=0)
    
    data = data.reset_index()
    data = data.drop(columns=['index', 'picture_link'])
    return data

'def import_recipe_data(): # import_recipes_main()\n    all_recipes = os.path.join(DATA_RECIPES_PATH, "recipes_raw_nosource_ar.json", orient=\'index\')\n    epicurious = os.path.join(DATA_RECIPES_PATH, "recipes_raw_nosource_epi.json", orient=\'index\')\n    food_network = os.path.join(DATA_RECIPES_PATH, "recipes_raw_nosource_fn.json", orient=\'index\')\n    \n    data =  pd.concat([all_recipes, epicurious, food_network], axis=0)\n    data = data.reset_index()\n    data = data.drop(columns=[\'picture_link\', \'index\'])\n    return data'

### D. Set Cuisine Classes

In [61]:
CUISINE_CLASSES = ['greek','southern_us','filipino','indian','jamaican','spanish','italian','mexican','chinese','british','thai','vietnamese','cajun_creole','brazilian','french','japanese','irish','korean','moroccan','russian']

## 2. Pre-Processing

### A. Stopwords

In [5]:
additional_stop_words = ["advertisement", "advertisements",
                         "cup", "cups",
                         "tablespoon", "tablespoons", 
                         "teaspoon", "teaspoons", 
                         "ounce", "ounces",
                         "salt", 
                         "pepper", 
                         "pound", "pounds",
                         ]

stopword_list = stopwords.words("english")
stopword_list.extend(additional_stop_words)

### B. String Cleaning

i. String Cleaning Function

In [80]:
"""def clean_string(list, lemmatize = True, stemming = False, lst_stopwords=None):
    str = ' '.join(list) #converting the list to string
    clean_text = ''
    
    lower = str.lower().split() #lowercase and tokenize
    
    lst_text=[]
    clean_words = []
    for word in lower:
        if len(word) > 2:
            digit = re.sub(r'\d+','', word) #removing digits
            text = re.sub(r'[^\w\s]', '', digit) #removing punc and characters            
            
            if lemmatize:
                lm = nltk.stem.wordnet.WordNetLemmatizer()  #lemmatize
                lemm = lm.lemmatize(text)
                clean_words.append(lemm)
                
                if stemming:
                    stemmer = nltk.stem.porter.PorterStemmer #stemming
                    stemm = stemmer.stem(text)
                    clean_words.append(stemm)
         
    #rem_stop = [i for i in clean_words if i not in stopword_list]  #remove stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
    
    clean_text = ' '.join(lst_text) #join as a string
    text = re.sub(' +', ' ', clean_text) #remove multi-spaces
    
    return text  """

In [88]:
# Adjust this function!!!
def clean_string(text, stemming=False, lemmatize=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
    if len(text) > 2:
        lst_text = text.split()

    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if stemming == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if lemmatize == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)

    ## Remove digits
    text = ''.join([i for i in text if not i.isdigit()])

    ## remove mutliple space
    text = re.sub(' +', ' ', text)

    return text

In [63]:
cuisine_df = import_cuisine_data()
clean_string(cuisine_df['ingredients'], lst_stopwords=stopword_list)

' romaine lettuce black olive grape tomato plain flour ground tomato g egg mayonaise cooking oil g water vegetable oil wheat black shallot cornflour cayenne pe light brown sugar granulated sugar butter kraft zesty italian dressing purple onion b egg citrus fruit raisin sourdough starte boneless chicken skinless thigh minced garli green chile jalapeno chilies onion ground name ingredient length dtype object'

ii. Preprocess Cuisine Data

In [89]:
def process_cuisine(): # !!!--- NEED TO UPDATE LATER IN CODE ---!!! originally: process_data()
    dataset = import_cuisine_data() # cuisine

    def processing(row):
        ls = row['ingredients']
        return ' '.join(ls)

    dataset['ingredients'] = dataset.apply(lambda x: processing(x), axis=1)
    dataset.dropna(inplace=True)
    dataset = dataset.drop(columns=['id']).reset_index(drop=True)

    dataset["ingredients_query"] = dataset["ingredients"].apply(lambda x: 
          clean_string(x, stemming=False, lemmatize=True, lst_stopwords=stopword_list)
          )
    return dataset

iii. Preprocess Recipes

In [82]:
def process_recipes(data): # Recipes dataset
    data["ingredients_query"] = data["ingredients"].apply(lambda x: 
            clean_string(x, stemming=False, lemmatize=True, lst_stopwords=stopword_list))
    return data

def get_tokenize_text(input_text):
    return clean_string(input_text, stemming=False, lemmatize=True, lst_stopwords=stopword_list)

### C. Word Embeddings

i. Word Embeddings from Cuisine Data

In [9]:
"""def create_embeddings(dataset):
    ## Tf-Idf (advanced variant of BoW)
    vectorizer = feature_extraction.text.TfidfVectorizer(max_features=10000, ngram_range=(1,2))

    corpus = dataset["ingredients_query"]
    vectorizer.fit(corpus)
    embedded_ingredients = vectorizer.transform(corpus)
    dic_vocabulary = vectorizer.vocabulary_

    ## Chi squarred correlation embeddings reduction
    labels = dataset["cuisine"]
    names = vectorizer.get_feature_names()
    p_value_limit = 0.95
    dtf_features = pd.DataFrame()

    for cat in np.unique(labels):
        chi2, p = feature_selection.chi2(embedded_ingredients, labels==cat)
        dtf_features = dtf_features.append(pd.DataFrame(
                       {"feature":names, "score":1-p, "labels":cat}))
        dtf_features = dtf_features.sort_values(["labels","score"], 
                        ascending=[True,False])
        dtf_features = dtf_features[dtf_features["score"]>p_value_limit]
    names = dtf_features["feature"].unique().tolist()

    ## Check the main ingredients
    for cat in np.unique(labels):
        print("# {}:".format(cat))
        print("  . selected features:",len(dtf_features[dtf_features["labels"]==cat]))
        print("  . top features:", ",".join(dtf_features[dtf_features["labels"]==cat]["feature"].values[:10]))
        print(" ")
    
    ## New embeddings
    vectorizer = feature_extraction.text.TfidfVectorizer(vocabulary=names)
    vectorizer.fit(corpus)
    embedded_ingredients = vectorizer.transform(corpus)
    dic_vocabulary = vectorizer.vocabulary_

    return vectorizer"""

## 3. Cuisine Prediction Model

### A. Tested Multiple Categorization Models

In [ ]:
# Sobanaa add here

### B. Using the best Model & Evaluating Performance Metrics

i. Saving Pkls

In [10]:
def save_pkl(file, pkl_filename):
    with open(pkl_filename, 'wb') as pkl_file:
        pickle.dump(file, pkl_file)

ii. Compute Performance Metrics

In [11]:
def compute_performances(predicted, predicted_prob, y_test):
    
    classes = np.unique(y_test)
    y_test_array = pd.get_dummies(y_test, drop_first=False).values

    ## Accuracy, Precision, Recall
    accuracy = metrics.accuracy_score(y_test, predicted)
    balance_accuracy = metrics.balanced_accuracy_score(y_test, predicted)
    auc = metrics.roc_auc_score(y_test, predicted_prob, 
                                multi_class="ovr")
    print("Balanced Accuracy:",  round(balance_accuracy,2))
    print("Accuracy:",  round(accuracy,2))
    print("Auc:", round(auc,2))
    print("Detail:")
    print(metrics.classification_report(y_test, predicted))

### B. Cuisine Prediction Model

i. Model Function

In [90]:
dataset = process_cuisine()

In [91]:
X = dataset['ingredients_query']
y = dataset['cuisine']

In [92]:
#TFIDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1, 2))

#Using stratified sampling
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, stratify = y, random_state = 123)

In [ ]:
"""#Fitting vectorizer
matrix_train_lr = vectorizer.fit_transform(X_train)
matrix_test_lr = vectorizer.transform(X_test)

#Fitting final model with Best hyper params
lr_final = LogisticRegression(max_iter = 300, random_state = 123, multi_class ='multinomial', solver = 'newton-cg', C = 10, penalty = 'l2')
lr_final.fit(matrix_train_lr, y_train)
y_pred = lr_final.predict(matrix_test_lr)
pred_prob = lr_final.predict_proba(matrix_test_lr)

#Classification metrics
print('f1 score weighted %s' % f1_score(y_test, y_pred, average = 'weighted'))
print('Accuracy score %s' % accuracy_score(y_test, y_pred))
#print('AUC score %s' % roc_auc_score(y_test, pred_prob))
#print('ROC score %s' % roc_curve(y_test, y_pred))
print(classification_report(y_test, y_pred))

#cm_lr_test=confusion_matrix(y_test, y_pred)
#print(cm_lr_test)

acc_lr = accuracy_score(y_test, y_pred)
f1_lr = f1_score(y_test, y_pred, average = 'weighted')"""

Calling Above:
- Vectorizer
- Split Dataset

Within Function:
- Classifier
- Fit vectorizer
- Fitting the model
- Saving as pkl

In [93]:
def create_model_cuisine_predictions():
    # Classifer
    classifier = LogisticRegression(random_state=123,
                                      max_iter=300,
                                      n_jobs=-1,
                                      multi_class = 'multinomial', 
                                      solver = 'newton-cg', 
                                      C = 10, 
                                      penalty = 'l2',
                                      verbose=1
                                      ) 
    ## pipeline
    model = pipeline.Pipeline([("vectorizer", vectorizer),  
                                ("classifier", classifier)])

    #Fitting Vectorizer
    #matrix_train_lr = vectorizer.fit_transform(X_train)
    #matrix_test_lr = vectorizer.transform(X_test)

    # Fit Model
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    pred_prob = model.predict_proba(X_train)

    # Save as Pkl
    save_pkl(model, os.path.join(MODEL_PATH, "pickle_model.pkl"))

ii. Check function

In [94]:
create_model_cuisine_predictions()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   21.0s finished


Define load pkl

In [58]:
## Load from file
def load_pkl(pkl_filename):
    with open(pkl_filename, 'rb') as pkl_file:
        return pickle.load(pkl_file)

iv. Database

In [57]:
MODEL_PATH = 'models/nlp'

def create_and_populate_db():
    data = import_recipe_data()
    
    # Process the data
    data = process_recipes(data)
    
    # Predict cuisine from trained model
    model = load_pkl(os.path.join(MODEL_PATH, 'pickle_model.pkl'))
    #data["cuisine"] = model.predict(data["ingredients_query"].tolist())
    
    db = sq.connect('recipes.db')
    #Verify dtypes
    for col in data.columns:
        data[col] = data[col].astype('str')

    print(' ------------------ Check data before populating the db ------------------')
    print(data.columns)
    print(data.head())
    print(data.shape)
    data.to_sql('main_recipes', db, if_exists='replace')

In [59]:
create_and_populate_db()

 ------------------ Check data before populating the db ------------------
Index(['title', 'ingredients', 'instructions', 'ingredients_query'], dtype='object')
                               title  \
0  Slow Cooker Chicken and Dumplings   
1      Awesome Slow Cooker Pot Roast   
2               Brown Sugar Meatloaf   
3        Best Chocolate Chip Cookies   
4  Homemade Mac and Cheese Casserole   

                                         ingredients  \
0  ['4 skinless, boneless chicken breast halves A...   
1  ['2 (10.75 ounce) cans condensed cream of mush...   
2  ['1/2 cup packed brown sugar ADVERTISEMENT', '...   
3  ['1 cup butter, softened ADVERTISEMENT', '1 cu...   
4  ['8 ounces whole wheat rotini pasta ADVERTISEM...   

                                        instructions  \
0  Place the chicken, butter, soup, and onion in ...   
1  In a slow cooker, mix cream of mushroom soup, ...   
2  Preheat oven to 350 degrees F (175 degrees C)....   
3  Preheat oven to 350 degrees F (175 

In [60]:
def get_df_from_db(cuisine):
    db = sq.connect('recipes.db')
    sql_query = "SELECT title, instructions, ingredients, ingredients_query FROM main_recipes WHERE cuisine = ?"
    return pd.read_sql(sql_query, db, params=(cuisine,))

v. Doc2Vec

In [43]:
def d2v_embeddings(data): # Recipe dataset
    data = data['ingredients_query'].tolist()
    tagged_data = [TaggedDocument(words=row.split(), tags=[str(index)]) for index, row in enumerate(data)]

    max_epochs = 20
    vec_size = 50
    alpha = 0.025

    model_embedding = Doc2Vec(vector_size=vec_size,
                        alpha=alpha, 
                        min_alpha=0.00025,
                        min_count=1,
                        dm =1)
  
    model_embedding.build_vocab(tagged_data)

    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model_embedding.train(tagged_data,
                    total_examples=model_embedding.corpus_count,
                    epochs=model_embedding.epochs)
        # decrease the learning rate
        model_embedding.alpha -= 0.0002
        # fix the learning rate, no decay
        model_embedding.min_alpha = model_embedding.alpha
    
    return model_embedding

In [62]:
def train_model_embeddings():
    db = sq.connect('recipes.db')
    cursor = db.cursor()
    
    for cuisine in CUISINE_CLASSES:
        sql_query = "SELECT title, instructions, ingredients, ingredients_query FROM main_recipes WHERE cuisine = ?"
        data = pd.read_sql(sql_query, db, params=(cuisine,))
        
        model_embedding = d2v_embeddings(data)
        save_pkl(model_embedding, os.path.join(MODEL_EMBEDDINGS_PATH, f'd2v_{cuisine}.pkl'))